# Exercice 1 :

###  1)

In [2]:
%%writefile hello_world_mpi.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

print("Hello world !")
    

Overwriting hello_world_mpi.py


In [3]:
!mpirun -n 4 python3  hello_world_mpi.py 

Hello world !
Hello world !
Hello world !
Hello world !


### 2)

In [4]:
%%writefile hello_world_mpi_r_s.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

def h_w():
    print("Hello world !",RANK,SIZE)
    
h_w()

Overwriting hello_world_mpi_r_s.py


In [5]:
!mpirun --oversubscribe -n 8 python3  hello_world_mpi.py #to use more than the real cores existing on the machine

Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !
Hello world !


### 3)

In [6]:
%%writefile hello_world_mpi_r_s_0.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

if RANK==0 :
    print("Hello world !",RANK,SIZE)

Overwriting hello_world_mpi_r_s_0.py


In [7]:
!mpirun -n 4 python3  hello_world_mpi.py 

Hello world !
Hello world !
Hello world !
Hello world !


# Exercice 2 :


In [7]:
%%writefile neg_value_exit.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

while True:
    if RANK == 0:
        # Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
    else:
        value = None
        
    # Send the value to all processes
    value = COMM.bcast(value, root=0)
    
    # Exit the program in case of negative input
    if value < 0:
        break

    
    # Print the received value and the rank of the current process
    print(f"Process {RANK} got {value}")
    COMM.Barrier()

In [8]:
!mpirun -n 4 python3  neg_value_exit.py

# Exercice 3 :

### 1)

In [16]:
%%writefile broadcast_by_ring.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

# Get the number of the machine cores
SIZE = COMM.Get_size()


while True:
    
    if RANK == 0:
        
        #Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
        COMM.send( value, dest= 1)
        
    
    else :
        
        value = COMM.recv( source= RANK-1)
        
        if RANK < SIZE-1 :
            COMM.send( value + RANK, dest= RANK + 1)
        
        
    if value < 0 :
        
        break
        
   
        
    # Print the received value and the rank of the current process
    
    print(f"Process {RANK} got {value}")
    COMM.Barrier()
        


        
        
        
        
        
        

# Exercice 4 :

In [1]:
import numpy as np
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
# define matrix dimensions
n = 8
m = 8
# create the matrix on process 0
if rank == 0:
    matrix = np.arange(n*m, dtype='float64').reshape(n, m)
    print("Original matrix on processor 0:")
    print(matrix)
else:
    matrix = None
# divide the matrix into parts and scatter to the other processes
sendcounts = [n//2 * m//2, n//2 * m//2, n//2 * m//2]
displs = [0, n//2 * m//2, 3*n//4 * m//2]
submatrix = np.empty((n//2, m//2), dtype='float64')
comm.Scatterv([matrix, sendcounts, displs, MPI.DOUBLE], submatrix, root=0)
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
if rank == 0:
    # Create a 4x4 matrix
    matrix = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]])
    print("Original matrix on processor 0:")
    print(matrix)
    # Define how many rows each processor should receive
    rows_per_proc = [2, 1, 1]
    # Define the displacements for each processor
    displacements = [0, 8, 12]
else:
    # Initialize empty matrix
    matrix = None
# Define rows_per_proc and displacements for non-zero processors
    rows_per_proc = None
    displacements = None
# Scatter parts of the matrix to different processors
d1_local = np.zeros(3)
rows = comm.Scatterv([matrix, rows_per_proc, displacements, MPI.INT],d1_local, root=0)
print(f"Received matrix on processor {rank}:")
print(rows)

from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
n = 8
m = 8
if rank == 0:
    n = 8
    m = 8
    A = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            A[i, j] = i * m + j + 1
    print("Original matrix on processor 0:")
    print(A)
# Divide the matrix into parts to send to each processor
    sendcounts = np.zeros(size, dtype=int)
    displs = np.zeros(size, dtype=int)
    sendcounts[1] = (n // 2) * (m - m // 2)
    sendcounts[2] = (n - n // 2) * (m // 2)
    sendcounts[3] = (n - n // 2) * (m - m // 2)
    displs[1] = (n // 2)
    displs[2] = (m //2) 
    displs[3] = (n - n // 2) * (m - m // 2)
else:
    A = None
    sendcounts = None
    displs = None
# Scatter the matrix parts to each processor
recvA = np.zeros((n // 2, m // 2))
recvcounts = (n // 2) * (m // 2)
print(np.transpose(A))
comm.Scatterv([np.transpose(A) , sendcounts, displs, MPI.DOUBLE], recvA, root=0)
if rank == 1:
    print("Received matrix on processor 1:")
    print(recvA)
elif rank == 2:
    print("Received matrix on processor 2:")
    print(recvA)
elif rank == 3:
    print("Received matrix on processor 2:")
    print(recvA)

NameError: name 'MPI' is not defined

# Exercice 5 :

In [ ]:
import numpy as np
import scipy
from scipy.sparse import lil_matrix
from numpy.random import rand, seed
from numba import njit
from mpi4py import MPI


''' This program compute parallel csc matrix vector multiplication using mpi '''

COMM = MPI.COMM_WORLD
size = COMM.Get_size()
RANK = COMM.Get_rank()

seed(42)

def matrixVectorMult(A, b, x, size):
    
    row, col = A.shape
    start = RANK * (row // size)
    end = start + (row // size)
   #x = np.zeros(col)
    for i in range(start, end):
       # a = A[i]
        for j in range(col):
            x[i] += A[i,j] * b[j]

    return x

########################initialize matrix A and vector b ######################
#matrix sizes
SIZE = 1000
#Local_size = 

# counts = block of each proc
#counts = 

if RANK == 0:
    A = lil_matrix((SIZE, SIZE))
    A[0, :100] = rand(100)
    A[1, 100:200] = A[0, :100]
    A.setdiag(rand(SIZE))
    A = A.toarray()
    b = rand(SIZE)
else :
    A = None
    b = None


#########Send b to all procs and scatter A (each proc has its own local matrix#####
LocalMatrix = lil_matrix((SIZE, SIZE))
# Scatter the matrix A
#COMM.Scatter(A, LocalMatrix, root=0)
#####################Compute A*b locally#######################################
b = np.zeros(SIZE)
Localb = np.zeros(SIZE) 
#LocalX = np.zeros(SIZE)
#COMM.Scatter(b, Localb, root=0)

LocalX = np.zeros(SIZE)
LocalX = matrixVectorMult(LocalMatrix, b, LocalX, size)
if RANK == 0:
    X = np.zeros(SIZE)

#recvcounts = np.full(4, SIZE // 4)
#displs = np.arange(4) * (SIZE // 4)
#COMM.Gatherv(LocalX, [X, recvcounts, displs, MPI.DOUBLE], root = 0)
# start = MPI.Wtime()
# matrixVectorMult(LocalMatrix, b, LocalX)
# stop = MPI.Wtime()

if RANK == 0:
    start = MPI.Wtime()
    matrixVectorMult(LocalMatrix, b, LocalX, size)
    stop = MPI.Wtime()
    print("CPU time of parallel multiplication is ", (stop - start))

##################Gather te results ###########################################
# sendcouns = local size of result
#sendcounts = 
if RANK == 0:
    X = matrixVectorMult(A, b, LocalX, size) 
else :
    X = None

# Gather the result into X


##################Print the results ###########################################

if RANK == 0 :
    start = MPI.Wtime()
    X_ = A.dot(b)
    stop = MPI.Wtime()
    print("The time to calculate A*b using dot is :", stop - start)
    # print("The result of A*b using parallel version is :", X)
    
    

if RANK == 0:
    import matplotlib.pyplot as plt
    plt.plot(X)
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.title('Parallel Matrix-Vector Multiplication')
    plt.show()

# Exercice 6 :

In [ ]:


from mpi4py import MPI
import math

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

N = 840

pi_part = 0.0
for i in range(rank, N, size):
    pi_part += 1.0/(1.0 + ((i + 0.5)/N)**2)
pi_part *= 4.0/N
#Print the computed value of pi for each process
print("Process", rank, "partial sum:", pi_part)
if rank == 0:
    pi = pi_part
    for i in range(1, size):
        pi += comm.recv(source=i)
else:
    comm.send(pi_part, dest=0)
if rank == 0:
    print("Computed pi:", pi)
comm.Barrier() 
t1 = MPI.Wtime()
num_runs = 10000 
for i in range(num_runs):
    pi_part = 0.0
    for j in range(rank, N, size):
        #Divide the iterations among the processes
        pi_part += 1.0/(1.0 + ((j + 0.5)/N)**2)
    pi_part *= 4.0/N
    #Collect and sum the partial results on the rank 0 process
    if rank == 0:
        pi = pi_part
        for j in range(1, size):
            pi += comm.recv(source=j)
    else:
        comm.send(pi_part, dest=0)
    if rank == 0 and i == 0:
        print("Computed pi:", pi)
comm.Barrier()
t2 = MPI.Wtime()
if rank == 0:
    print("Taken Time:", t2 - t1)